In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium

Requesting URL and getting the content

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Mon, 01 Jun 2020 03:17:45 GMT', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Server': 'ATS/8.0.7', 'Content-Type': 'text/html; charset=UTF-8', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Last-Modified': 'Sat, 23 May 2020 19:32:34 GMT', 'Content-Encoding': 'gzip', 'Age': '43042', 'X-Cache': 'cp5008 miss, cp5012 hit/28', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=01-Jun-2020;Path=/;HttpOnly;secure;Expires=Fri, 03 Jul 2020 12:00:00 GMT, WMF-Last-Access-Global=01-Jun-2020;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Fri, 03 Jul 2020 12:00:00 GMT, GeoIP=IN:TG:Hyderabad:17.38:78.46:v4; Path=/; secure; Domain=.wikipedia.org', 'X-Client-IP': '183.83.37.151'

Cleaning with beautiful soup

In [3]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
        
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

In [4]:

cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
 

Combining Neighborhoods

In [6]:
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first',         
        'Neighborhood': ', '.join,}
    ).reset_index()

In [7]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Shape

In [8]:
df.shape

(103, 3)

Will read geo data from a CSV


In [9]:
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

Need to merge 2 dataframes based on a PostalCode column with example

In [11]:

df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')
df2.loc[df2['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [13]:

df2.loc[df2['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
